In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as	plt
import seaborn as sns
import json
import math

# Part 1

In [2]:
df = pd.read_csv('train.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

C:\Users\Admin\AppData\Local\Temp\ipykernel_14452\2090610779.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('train.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])


In [3]:
display(df)

,Token,Label
0,Estuvimos,O
1,hace,O
2,poco,O
3,mi,O
4,pareja,O
...,...,...
31171,junto,O
31172,con,O
31173,la,O
31174,carta,O


In [4]:
df_token = df.loc[:,["Token"]]
df_label = df.loc[:,["Label"]]

arr_token = (df_token.squeeze()).to_numpy()
arr_label = (df_label.squeeze()).to_numpy()

token_all = df_token.squeeze().unique()
label_all = df_label.squeeze().unique()

In [5]:
print(label_all)

['O' 'B-positive' 'B-negative' 'B-neutral' 'I-neutral' 'I-positive'
 'I-negative']


In [6]:
# y_all = pd.DataFrame(df_label.value_counts()).reset_index()
# y_all.columns = ['Label', 'Count']
# y_all["Location"] = [np.where(arr_label == i)[0] for i in label_all]

y_all = pd.DataFrame(label_all, columns=['Label'])
y_all["Location"] = [np.where(arr_label == i)[0] for i in label_all]
y_all["Count"] = [len(y_all.loc[i, "Location"]) for i in range(len(y_all))]

print(len(y_all))
print(type(y_all))
display(y_all)

7
<class 'pandas.core.frame.DataFrame'>


,Label,Location,Count
0,O,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",29035
1,B-positive,"[16, 19, 22, 25, 60, 63, 75, 86, 90, 113, 204,...",1160
2,B-negative,"[39, 98, 124, 131, 133, 137, 323, 382, 472, 62...",381
3,B-neutral,"[153, 591, 685, 1053, 1180, 1467, 1973, 1994, ...",72
4,I-neutral,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...",43
5,I-positive,"[313, 314, 315, 937, 1400, 1401, 1939, 2054, 2...",314
6,I-negative,"[802, 1171, 1626, 1627, 1724, 1725, 1726, 1727...",171


In [7]:
x_all = pd.DataFrame(token_all, columns=['Token'])

# Need to check more, this takes >1m to do
new_x_all = pd.DataFrame({'Token': np.repeat(x_all['Token'], 7)}).reset_index()
new_x_all.columns = ['Index', 'Token']
new_x_all["Label"] = np.tile(label_all, int(len(new_x_all)/7))
count_x_all = []
location_x_all = []

for i in token_all:
    for j in label_all:
        cond_1 = arr_token == i
        cond_2 = arr_label == j
        loc = (np.where(cond_1 & cond_2)[0]).tolist()
        # df_ij = df.iloc[loc, :]
        count_x_all += [len(loc)]
        location_x_all += [loc]
new_x_all["Count"] = count_x_all
new_x_all["Location"] = location_x_all

print(len(new_x_all))
print(type(new_x_all))
display(new_x_all)

35280
<class 'pandas.core.frame.DataFrame'>


,Index,Token,Label,Count,Location
0,0,Estuvimos,O,6,"[0, 13026, 13936, 17095, 27870, 29879]"
1,0,Estuvimos,B-positive,0,[]
2,0,Estuvimos,B-negative,0,[]
3,0,Estuvimos,B-neutral,0,[]
4,0,Estuvimos,I-neutral,0,[]
...,...,...,...,...,...
35275,5039,inolvidable,B-negative,0,[]
35276,5039,inolvidable,B-neutral,0,[]
35277,5039,inolvidable,I-neutral,0,[]
35278,5039,inolvidable,I-positive,0,[]


$1. \:\:\: e(x \mid y)=\frac{\operatorname{Count}(y \rightarrow x)}{\operatorname{Count}(y)}$

In [8]:
def emm_par(x, y):
    # x is an instance of Token
    # y is an instance of Label
    cond_1 = new_x_all["Token"] == x
    cond_2 = new_x_all["Label"] == y
    num = ((new_x_all[cond_1 & cond_2])["Count"]).values[0]
    cond_3 = y_all["Label"] == y
    denom = ((y_all[cond_3])["Count"]).values[0]
    return num/denom

In [9]:
a = emm_par("Estuvimos", "O")
print(a)

b = emm_par("estuvimos", "O")
print(b)

0.0002066471499913897
0.00030997072498708454


$2. \:\:\:e(x \mid y)=\left\{\begin{array}{cc}
\frac{\operatorname{count}(y \rightarrow x)}{\operatorname{Count}(y)+k} & \text { If the word token } x \text { appears in the training set } \\
\frac{k}{\operatorname{Count}(y)+k} & \text { If word token } x \text { is the special token \#UNK\# }
\end{array}\right.$

In [10]:
def emm_par_test(x, y):
    # x is an instance of Token
    # y is an instance of Label
    k = 1
    if x in new_x_all.loc[:, "Token"].unique():
        cond_1 = new_x_all["Token"] == x
        cond_2 = new_x_all["Label"] == y
        num = ((new_x_all[cond_1 & cond_2])["Count"]).values[0]
    else:
        num = k
   git cond_3 = y_all["Label"] == y
    denom = ((y_all[cond_3])["Count"]).values[0] + k
    return num/denom

In [11]:
a = emm_par_test(",", "I-negative")
print(a)

b = emm_par_test("Kontollllllllllll", "I-neutral")
print(b)

0.01744186046511628
0.022727272727272728


In [12]:
emm_par_score_all = []
new_token_all = np.append(token_all, "#UNK#")
print(new_token_all)

['Estuvimos' 'hace' 'poco' ... 'redondeó' 'inolvidable' '#UNK#']


In [13]:
# Need to check more, this takes >2m to do
for i in token_all:
    emm_par_score = 0
    for j in label_all:
        emm_par_score = emm_par_test(i,j)
        emm_par_score_all += [emm_par_score]
print(emm_par_score_all)

[0.00020664003306240529, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0008954401432704229, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0018942003030720485, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0024796803967488635, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00044772007163521146, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0352665656426505, 0.0, 0.0, 0.0, 0.0, 0.015873015873015872, 0.011627906976744186, 0.0012398401983744318, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005813953488372093, 0.00034440005510400884, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00013776002204160352, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0039606006336961016, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005813953488372093, 0.013638242182118749, 0.0, 0.0, 0.0, 0.0, 0.006349206349206349, 0.0, 0.005682600909216145, 0.0008613264427217916, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05730816916930707, 0.0, 0.0, 0.0, 0.0, 0.006349206349206349, 0.01744186046511628, 0.0013431602149056344, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.026002204160352666, 0.0, 0.0, 0.0, 0.022727272727272728, 0.0380952380952381, 0.01744186046511628, 0.0011020801763328282, 0

In [14]:
new_x_all["e(x|y)"] = emm_par_score_all
display(new_x_all)

,Index,Token,Label,Count,Location,e(x|y)
0,0,Estuvimos,O,6,"[0, 13026, 13936, 17095, 27870, 29879]",0.000207
1,0,Estuvimos,B-positive,0,[],0.000000
2,0,Estuvimos,B-negative,0,[],0.000000
3,0,Estuvimos,B-neutral,0,[],0.000000
4,0,Estuvimos,I-neutral,0,[],0.000000
...,...,...,...,...,...,...
35275,5039,inolvidable,B-negative,0,[],0.000000
35276,5039,inolvidable,B-neutral,0,[],0.000000
35277,5039,inolvidable,I-neutral,0,[],0.000000
35278,5039,inolvidable,I-positive,0,[],0.000000


$3. \:\:\:y^*=\underset{y}{\arg \max } \:e(x \mid y)$

In [15]:
df_y = pd.DataFrame()

emm_par_score_all = []
y_star_all = []
new_token_all = np.append(token_all, "#UNK#")
print(new_token_all)
print(len(new_token_all))

['Estuvimos' 'hace' 'poco' ... 'redondeó' 'inolvidable' '#UNK#']
5041


In [16]:
# Need to check more, this takes >2m to do
for i in new_token_all:
    emm_par_score = 0
    y_star = ""
    for j in label_all:
        emm_par_score_temp = emm_par_test(i,j)
        if emm_par_score_temp > emm_par_score:
            emm_par_score = emm_par_score_temp
            y_star = j
        else:
            pass
    y_star_all += [y_star]
    emm_par_score_all += [emm_par_score]

In [17]:
print(y_star_all)
print(len(y_star_all))
print(emm_par_score_all)
print(len(emm_par_score_all))

['O', 'O', 'O', 'O', 'O', 'O', 'I-negative', 'O', 'O', 'I-negative', 'O', 'O', 'O', 'O', 'I-positive', 'B-positive', 'I-neutral', 'B-negative', 'B-positive', 'B-positive', 'B-negative', 'O', 'O', 'I-negative', 'O', 'O', 'O', 'O', 'I-neutral', 'O', 'I-negative', 'O', 'O', 'O', 'I-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-negative', 'O', 'O', 'B-neutral', 'O', 'B-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'B-negative', 'O', 'O', 'B-positive', 'O', 'O', 'O', 'O', 'O', 'O', 'B-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-positive', 'O', 'I-positive', 'B-negative', 'I-neutral', 'O', 'O', 'B-negative', 'B-negative', 'O', 'O', 'B-negative', 'O', 'O', 'O', 'I-neutral', 'O', 'I-positive', 'O', 'B-neutral', 'I-neutral', 'I-neutral', 'I-neutral', 'I-neutral', 'I-neutral', 'I-neutral', 'O', 'O', 'I-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-neutral', 'O', 'I-negative', 'I-negative', 'O', 'O',

In [18]:
df_y["Token"] = new_token_all
df_y["y*"] = y_star_all
df_y["Max e(x|y)"] = emm_par_score_all

display(df_y)

,Token,y*,Max e(x|y)
0,Estuvimos,O,0.000207
1,hace,O,0.000895
2,poco,O,0.001894
3,mi,O,0.002480
4,pareja,O,0.000448
...,...,...,...
5036,informado,O,0.000034
5037,almuerzo,O,0.000034
5038,redondeó,O,0.000034
5039,inolvidable,O,0.000034


In [19]:
df_y_dict = {row['Token']: (row['y*'], row['Max e(x|y)']) for row in df_y.to_dict(orient='records')}
print(df_y_dict)

{'Estuvimos': ('O', 0.00020664003306240529), 'hace': ('O', 0.0008954401432704229), 'poco': ('O', 0.0018942003030720485), 'mi': ('O', 0.0024796803967488635), 'pareja': ('O', 0.00044772007163521146), 'y': ('O', 0.0352665656426505), 'yo': ('I-negative', 0.005813953488372093), 'comiendo': ('O', 0.00034440005510400884), 'resultó': ('O', 0.00013776002204160352), 'todo': ('I-negative', 0.005813953488372093), 'muy': ('O', 0.013638242182118749), 'bien': ('O', 0.005682600909216145), ',': ('O', 0.05730816916930707), 'tanto': ('O', 0.0013431602149056344), 'la': ('I-positive', 0.0380952380952381), 'comida': ('B-positive', 0.14556416881998277), 'el': ('I-neutral', 0.022727272727272728), 'vino': ('B-negative', 0.015706806282722512), 'trato': ('B-positive', 0.03789836347975883), 'decoración': ('B-positive', 0.006029285099052541), '…': ('B-negative', 0.002617801047120419), 'nos': ('O', 0.005028240804518529), 'gustó': ('O', 0.0003788400606144097), 'mucho': ('I-negative', 0.005813953488372093), '.': ('O'

### Testing

In [30]:
import pandas as pd

# Specify the correct encoding for the file (e.g., utf-8)
file_encoding = 'utf-8'

# Read the data line by line
with open('dev_out.txt', 'r', encoding=file_encoding) as f:
    lines = f.read().splitlines()

# Split each line into two parts and replace empty lines with [np.nan, np.nan]
data = [line.split(' ') if line else ["", ""] for line in lines]

# Convert the list of lists into a DataFrame
df_test = pd.DataFrame(data, columns=['Token', 'Label'])

# Display the DataFrame
print(df_test)

            Token       Label
0           Plato  B-positive
1     degustación  I-positive
2               :           O
3              un           O
4            poco           O
...           ...         ...
4573      dejarte           O
4574           la           O
4575      cartera           O
4576            .           O
4577                         

[4578 rows x 2 columns]


In [31]:
display(df_test)

,Token,Label
0,Plato,B-positive
1,degustación,I-positive
2,:,O
3,un,O
4,poco,O
...,...,...
4573,dejarte,O
4574,la,O
4575,cartera,O
4576,.,O


In [36]:
df_test_token = df_test.loc[:,["Token"]]
df_test_label = df_test.loc[:,["Label"]]

arr_test_token = (df_test_token.squeeze()).to_numpy()
arr_test_label = (df_test_label.squeeze()).to_numpy()

token_test_all = df_test_token.squeeze().unique()
label_test_all = df_test_label.squeeze().unique()

In [37]:
test_pred = []
new_token = []
for i in arr_test_token:
    try:
        pred = df_y_dict[i][0]
        test_pred += [pred]
    except:
        if not i:
            pred = ""
        else:
            pred = df_y_dict["#UNK#"][0]    
        test_pred += [pred]
        new_token += [i]

In [38]:
df_test["Predicted Label"] = test_pred
display(df_test)

arr_test_pred_label = ((df_test.loc[:,["Predicted Label"]]).squeeze()).to_numpy()

,Token,Label,Predicted Label
0,Plato,B-positive,B-negative
1,degustación,I-positive,I-positive
2,:,O,O
3,un,O,O
4,poco,O,O
...,...,...,...
4573,dejarte,O,I-neutral
4574,la,O,I-positive
4575,cartera,O,I-neutral
4576,.,O,O


In [40]:
df_out = df_test.loc[:, ["Token", "Predicted Label"]]
df_out.to_csv('dev.prediction', sep=' ', index=False, header=False)

# Part 2

In [44]:
import pandas as pd

# Read the data line by line
with open('train.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

# Split each line into two parts and replace empty lines with [np.nan, np.nan]
data = [line.split(' ') if line.strip() else ["", ""] for line in lines]

# Convert the list of lists into a DataFrame
df_ss = pd.DataFrame(data, columns=['Token', 'Label (yi-1)'])

# Display the DataFrame
print(df_ss)


            Token Label (yi-1)
0           Plato   B-positive
1     degustación   I-positive
2               :            O
3              un            O
4            poco            O
...           ...          ...
4573      dejarte            O
4574           la            O
4575      cartera            O
4576            .            O
4577                          

[4578 rows x 2 columns]


In [45]:
display(df_ss)

,Token,Label (yi-1)
0,Plato,B-positive
1,degustación,I-positive
2,:,O
3,un,O
4,poco,O
...,...,...
4573,dejarte,O
4574,la,O
4575,cartera,O
4576,.,O


In [46]:
df_label_ss = df_ss.loc[:,["Label (yi-1)"]]
arr_label_ss = (df_label_ss.squeeze()).to_numpy()

In [47]:
label_all_ss = np.insert(label_all, 0, 'START')
label_all_ss = np.append(label_all_ss, 'STOP')

In [48]:
y_all_ss = pd.DataFrame(label_all_ss, columns=['Label (yi-1)'])

In [49]:
display(df_label_ss)
display(arr_label_ss)
display(label_all_ss)
display(y_all_ss)

,Label (yi-1)
0,B-positive
1,I-positive
2,O
3,O
4,O
...,...
4573,O
4574,O
4575,O
4576,O


array(['B-positive', 'I-positive', 'O', ..., 'O', 'O', ''], dtype=object)

array(['START', 'O', 'B-positive', 'B-negative', 'B-neutral', 'I-neutral',
       'I-positive', 'I-negative', 'STOP'], dtype=object)

,Label (yi-1)
0,START
1,O
2,B-positive
3,B-negative
4,B-neutral
5,I-neutral
6,I-positive
7,I-negative
8,STOP


In [50]:
new_x_all_ss = y_all_ss.copy()
new_x_all_ss.columns = ["Label"]
display(new_x_all_ss)

,Label
0,START
1,O
2,B-positive
3,B-negative
4,B-neutral
5,I-neutral
6,I-positive
7,I-negative
8,STOP


In [51]:
new_y_all_ss = pd.DataFrame({'Label (yi-1)': np.repeat(y_all_ss['Label (yi-1)'], 9)}).reset_index()
new_y_all_ss.columns = ['Index', 'Label (yi-1)']
new_y_all_ss["Destination Label (yi)"] = np.tile(label_all_ss, int(len(new_y_all_ss)/9))
new_y_all_ss["Count (yi-1,yi)"] = np.zeros(len(new_y_all_ss), dtype=int)

previous_y = "START"
for i in arr_label_ss:
    if i == "":
        i = "STOP"
    current_y = i
    new_y_all_ss.loc[(new_y_all_ss['Label (yi-1)'] == previous_y) & (new_y_all_ss['Destination Label (yi)'] == current_y), 'Count (yi-1,yi)'] += 1
    if current_y == "STOP":
        current_y = "START"
    previous_y = current_y

display(new_y_all_ss)

,Index,Label (yi-1),Destination Label (yi),"Count (yi-1,yi)"
0,0,START,START,0
1,0,START,O,244
2,0,START,B-positive,18
3,0,START,B-negative,4
4,0,START,B-neutral,0
...,...,...,...,...
76,8,STOP,B-neutral,0
77,8,STOP,I-neutral,0
78,8,STOP,I-positive,0
79,8,STOP,I-negative,0


In [52]:
y_all_ss_sums = [int(new_y_all_ss.iloc[i:i+9]['Count (yi-1,yi)'].sum()) for i in range(0, len(new_y_all_ss), 9)]
print(y_all_ss_sums)

new_x_all_ss["Count"] = y_all_ss_sums
new_x_all_ss.loc[new_x_all_ss["Label"] == "STOP", "Count"] = new_x_all_ss.loc[new_x_all_ss["Label"]=="START", "Count"].values[0]
display(new_x_all_ss)

[266, 3998, 160, 61, 8, 0, 49, 36, 0]


,Label,Count
0,START,266
1,O,3998
2,B-positive,160
3,B-negative,61
4,B-neutral,8
5,I-neutral,0
6,I-positive,49
7,I-negative,36
8,STOP,266


$2. \:\:\:q\left(y_i \mid y_{i-1}\right)=\frac{\operatorname{Count}\left(y_{i-1}, y_i\right)}{\operatorname{Count}\left(y_{i-1}\right)}$

In [53]:
def q_test(x, y):
    cond_1 = new_y_all_ss["Label (yi-1)"] == y
    cond_2 = new_y_all_ss["Destination Label (yi)"] == x
    num = ((new_y_all_ss[cond_1 & cond_2])["Count (yi-1,yi)"]).values[0]
    cond_3 = new_x_all_ss["Label"] == y
    denom = ((new_x_all_ss[cond_3])["Count"]).values[0]
    if not denom:
        return 0
    return num/denom

In [54]:
q_value_all = []
for i in label_all_ss:
    for j in label_all_ss:
        # print(i,j)
        q_value = q_test(j,i)
        q_value_all += [q_value]
print(q_value_all)

[0.0, 0.9172932330827067, 0.06766917293233082, 0.015037593984962405, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8819409704852427, 0.03551775887943972, 0.014257128564282141, 0.002001000500250125, 0.0, 0.0, 0.0, 0.0662831415707854, 0.0, 0.84375, 0.0, 0.0, 0.0, 0.0, 0.15625, 0.0, 0.0, 0.0, 0.7868852459016393, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21311475409836064, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.4897959183673469, 0.0, 0.0, 0.0, 0.0, 0.4897959183673469, 0.0, 0.02040816326530612, 0.0, 0.3611111111111111, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6388888888888888, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [55]:
new_y_all_ss["q(yi|yi-1)"] = q_value_all


In [56]:
display(new_y_all_ss)

,Index,Label (yi-1),Destination Label (yi),"Count (yi-1,yi)",q(yi|yi-1)
0,0,START,START,0,0.000000
1,0,START,O,244,0.917293
2,0,START,B-positive,18,0.067669
3,0,START,B-negative,4,0.015038
4,0,START,B-neutral,0,0.000000
...,...,...,...,...,...
76,8,STOP,B-neutral,0,0.000000
77,8,STOP,I-neutral,0,0.000000
78,8,STOP,I-positive,0,0.000000
79,8,STOP,I-negative,0,0.000000


In [57]:
display(new_x_all)

,Index,Token,Label,Count,Location,e(x|y)
0,0,Estuvimos,O,6,"[0, 13026, 13936, 17095, 27870, 29879]",0.000207
1,0,Estuvimos,B-positive,0,[],0.000000
2,0,Estuvimos,B-negative,0,[],0.000000
3,0,Estuvimos,B-neutral,0,[],0.000000
4,0,Estuvimos,I-neutral,0,[],0.000000
...,...,...,...,...,...,...
35275,5039,inolvidable,B-negative,0,[],0.000000
35276,5039,inolvidable,B-neutral,0,[],0.000000
35277,5039,inolvidable,I-neutral,0,[],0.000000
35278,5039,inolvidable,I-positive,0,[],0.000000


$3. \:\:\:y_1^*, \ldots, y_n^*=\underset{y_1, \ldots, y_n}{\arg \max } \:\:p\left(x_1, \ldots, x_n, y_1, \ldots, y_n\right)$

In [58]:
def viterbi(observed_states, hidden_states, start_prob, trans_prob, emit_prob, all_states):
    # Initialize V (viterbi variable) and backpointers
    V = [{}]
    backpointers = [{}]

    # Initialization step
    for y in hidden_states:
        if observed_states[0] not in all_states:
            observed_states[0] = 'I-neutral'
        V[0][y] = start_prob[y] * emit_prob[y].get(observed_states[0], 0)
        backpointers[0][y] = None

    # Recursion step
    for t in range(1, len(observed_states)):
        V.append({})
        backpointers.append({})
        if observed_states[t] not in all_states:
            observed_states[t] = 'I-neutral'
        for y in hidden_states:
            # Calculate the maximum probability and the state that achieved it
            max_prob, best_prev_state = max((V[t-1][y0] * trans_prob[y0].get(y, 0) * emit_prob[y].get(observed_states[t], 0), y0) for y0 in hidden_states)
            V[t][y] = max_prob
            backpointers[t][y] = best_prev_state

    # Final transition to STOP state
    V.append({})
    backpointers.append({})
    final_hidden_states = [state for state in hidden_states if state != 'START']
    for y in final_hidden_states:
        V[-1][y] = V[-2][y] if y in V[-2] else 0
    max_prob, best_prev_state = max((V[-1][y] * trans_prob[y].get('STOP', 0), y) for y in final_hidden_states)
    V[-1]['STOP'] = max_prob
    backpointers[-1]['STOP'] = best_prev_state

    # Backtrack to find best path
    best_path = ['STOP']
    for t in range(len(backpointers) - 1, 0, -1):
        best_path.insert(0, backpointers[t][best_path[0]])

    return best_path

##PART 3

In [350]:

def kth(k, observations, hidden_states, start_prob, trans_prob, emit_prob, all_states):
    T = len(observations)
    
    # Initialize a list to store the K-best sequences and their probabilities
    k_best_sequences = [([], 1.0)]
    
    # Find the best output sequence using the Viterbi algorithm
    best_sequence = viterbi(observations, hidden_states, start_prob, trans_prob, emit_prob,all_states)
    k_best_sequences.append((best_sequence, 1.0))
    
    # Apply Yen's algorithm to find K-1 deviations of the best sequence
    for i in range(1, k):
        # Find the subsequence to be deviated from the best sequence
        subsequence_to_deviate = best_sequence[:i]
        
        # Find the K-th best output sequence that includes the deviation using the Viterbi algorithm
        deviation_sequence = viterbi(observations[i:], hidden_states, start_prob, trans_prob, emit_prob,all_states)
        
        # Combine the common prefix and the deviation sequence
        kth_best_sequence = subsequence_to_deviate + deviation_sequence
        
        # Calculate the probability of the K-th best sequence
        kth_best_prob = 1.0
        for t in range(T):
            state = kth_best_sequence[t]
            observation = observations[t]
            kth_best_prob *= emit_prob[state].get(observation, 0)
            if t > 0:
                prev_state = kth_best_sequence[t - 1]
                kth_best_prob *= trans_prob[prev_state].get(state, 0)
        
        # Add the K-th best sequence and its probability to the list
        k_best_sequences.append((kth_best_sequence, kth_best_prob))
    
    # Sort the K-best sequences by probability and return the K-th best sequence
    k_best_sequences.sort(reverse=True, key=lambda x: x[1])
    return k_best_sequences[k - 1][0]


### Testing

In [337]:
def create_start_prob(df1, df2):
    count_values = df1.loc[df1['Label (yi-1)'] == 'START', 'Count (yi-1,yi)'].tolist()
    count_start = df2.loc[df2['Label'] == 'START', 'Count'].values[0]
    list1 = [value / count_start for value in count_values]
    list2 = label_all_ss.tolist()
    return dict(zip(list2, list1))

def create_trans_prob(df):
    trans_prob = {}
    for index, row in df.iterrows():
        yi_1 = row['Label (yi-1)']
        yi = row['Destination Label (yi)']
        prob = row['q(yi|yi-1)']
        if yi_1 not in trans_prob:
            trans_prob[yi_1] = {}
        trans_prob[yi_1][yi] = prob
    return trans_prob

def create_emit_prob(df):
    emit_prob = {}
    for index, row in df.iterrows():
        label = row['Label']
        token = row['Token']
        prob = row['e(x|y)']
        if label not in emit_prob:
            emit_prob[label] = {}
        emit_prob[label][token] = prob
    # Add "START" and "STOP" with all zero probabilities
    emit_prob["START"] = {token: 0 for token in df['Token'].unique()}
    emit_prob["STOP"] = {token: 0 for token in df['Token'].unique()}
    return emit_prob

def create_observed_states(df):
    big_list = []
    small_list = []
    for token in df['Token']:
        if token.strip() != '':
            small_list.append(token)
        else:
            big_list.append(small_list)
            small_list = []
    return big_list

In [338]:
all_states = token_all.tolist()
print(all_states)

observed_states = create_observed_states(df_test)
print(observed_states)

hidden_states = label_all_ss.tolist()
print(hidden_states)

start_prob = create_start_prob(new_y_all_ss, new_x_all_ss)
print(start_prob)

trans_prob = create_trans_prob(new_y_all_ss)
print(trans_prob)

emit_prob = create_emit_prob(new_x_all)
print(emit_prob)

['Estuvimos', 'hace', 'poco', 'mi', 'pareja', 'y', 'yo', 'comiendo', 'resultó', 'todo', 'muy', 'bien', ',', 'tanto', 'la', 'comida', 'el', 'vino', 'trato', 'decoración', '…', 'nos', 'gustó', 'mucho', '.', 'Por', 'poner', 'algún', 'pero', 'quizá', 'jamón', 'no', 'era', 'lo', '"', 'ibérico', 'que', 'cabía', 'esperar', 'Bien', 'sabe', 'autor', 'del', 'blog', ')', 'Comida', 'exquisita', 'Restaurante', 'diferente', 'creativo', 'agradable', 'Si', 'has', 'probado', 'sus', 'carnes', 'te', 'estas', 'perdiendo', 'algo', 'grande', '!', 'En', 'resumen', 'bien-muy', 'servicio', 'correcto', 'profesional', '02-12-', '2012', 'elegimos', 'este', 'restaurante', 'por', 'los', 'comentarios', 'ha', 'sido', 'una', 'Grandisima', 'Decepción', 'Salimos', 'encantadas', 'Ubicación', 'Comimos', 'atendidos', 'Los', 'servicios', 'en', 'estado', 'lamentable', 'manteles', 'copas', 'etc', '<->', 'Gambas', 'resecas', 'sabemos', 'llenaron', 'con', 'trozos', 'de', 'esa', 'sucedanio', 'forma', 'palo', '-', 'sabor', 'a', '

In [339]:
thing = observed_states[0]
print(thing)
type(thing)

['Plato', 'degustación', ':', 'un', 'poco', 'abundante', 'de', 'más', ',', 'pero', 'bien', 'cocinado', '.']


list

In [340]:
a = viterbi(thing, hidden_states, start_prob, trans_prob, emit_prob, all_states)
print(a)




['B-negative', 'I-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'STOP']


In [351]:
hello = kth(8, thing, hidden_states, start_prob, trans_prob, emit_prob, all_states)
print(hello)

['B-negative', 'I-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'STOP']


In [309]:

def read_input(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]

def write_output(file_path, outputs):
    with open(file_path, 'w', encoding='utf-8') as f:
        for output in outputs:
            f.write(output + '\n')


es_dev_input = read_input('dev_in.txt')


# Apply k-best Viterbi for ES
es_k_best_2nd = kth_best_output_viterbi(2,es_dev_input, hidden_states, start_prob, trans_prob, emit_prob,all_states)
es_k_best_8th = get_k_best_sequences(es_dev_input, hidden_states, start_prob, trans_prob, emit_prob, all_states, 8)[7]

# Write outputs to files
# write_output('RU/dev.p3.2nd.out', ru_k_best_2nd)
# write_output('RU/dev.p3.8th.out', ru_k_best_8th)

write_output('dev.p4.2nd.out', ru_k_best_2nd)
write_output('dev.p4.8th.out', ru_k_best_8th)

['Estuvimos', 'hace', 'poco', 'mi', 'pareja', 'y', 'yo', 'comiendo', 'resultó', 'todo', 'muy', 'bien', ',', 'tanto', 'la', 'comida', 'el', 'vino', 'trato', 'decoración', '…', 'nos', 'gustó', 'mucho', '.', 'Por', 'poner', 'algún', 'pero', 'quizá', 'jamón', 'no', 'era', 'lo', '"', 'ibérico', 'que', 'cabía', 'esperar', 'Bien', 'sabe', 'autor', 'del', 'blog', ')', 'Comida', 'exquisita', 'Restaurante', 'diferente', 'creativo', 'agradable', 'Si', 'has', 'probado', 'sus', 'carnes', 'te', 'estas', 'perdiendo', 'algo', 'grande', '!', 'En', 'resumen', 'bien-muy', 'servicio', 'correcto', 'profesional', '02-12-', '2012', 'elegimos', 'este', 'restaurante', 'por', 'los', 'comentarios', 'ha', 'sido', 'una', 'Grandisima', 'Decepción', 'Salimos', 'encantadas', 'Ubicación', 'Comimos', 'atendidos', 'Los', 'servicios', 'en', 'estado', 'lamentable', 'manteles', 'copas', 'etc', '<->', 'Gambas', 'resecas', 'sabemos', 'llenaron', 'con', 'trozos', 'de', 'esa', 'sucedanio', 'forma', 'palo', '-', 'sabor', 'a', '

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
# entity_loc = (np.where(arr_test_label != "O")[0]).tolist()
# pred_entity_loc = (np.where(arr_test_pred_label != "O")[0]).tolist()

In [ ]:
# print(entity_loc)
# print(pred_entity_loc)

In [ ]:
# total_num_pred_e = len(pred_entity_loc)
# total_num_gold_e = len(entity_loc)

# df_test_entity = ((df_test.loc[entity_loc,:]).copy()).reset_index(drop=True)
# display(df_test_entity)
# df_test_entity_label = df_test_entity.loc[:,["Label"]]
# df_test_entity_pred_label = df_test_entity.loc[:,["Predicted Label"]]

# c_pred_entity_loc = (np.where((df_test_entity_label.squeeze()).to_numpy() == (df_test_entity_pred_label.squeeze()).to_numpy())[0]).tolist()
# print(c_pred_entity_loc)

# total_num_c_pred_e = len(c_pred_entity_loc)

In [ ]:
# print(total_num_pred_e)
# print(total_num_gold_e)
# print(total_num_c_pred_e)

$\text { Precision }=\frac{\text { Total number of correctly predicted entities }}{\text { Total number of predicted entities }}$

In [ ]:
# precision = total_num_c_pred_e/total_num_pred_e

$\text { Recall }=\frac{\text { Total number of correctly predicted entities }}{\text { Total number of gold entities }}$

In [ ]:
# recall = total_num_c_pred_e/total_num_gold_e

$F=\frac{2}{1 / \text { Precision }+1 / \text { Recall }}$

In [ ]:
# f_score = 2/((1/precision)+(1/recall))

In [ ]:
# print(precision)
# print(recall)
# print(f_score)